# Retrieval Augmented Question (RAG) Application with Llama3-8B on SageMaker JumpStart using LangChain

RAG Application use cases with Llama3-8B on SageMaker Jumpstart

In this notebook, we demonstrate the use of [Llama3-8B](https://huggingface.co/meta-llama/Llama-2-13b) text generation combined with [BGE Large En v1.5](https://huggingface.co/BAAI/bge-large-en-v1.5) embedding model to efficiently construct a Retrieval Augmented Generation (RAG) QnA system on a SageMaker Notebook. This notebook, powered by an `ml.t3.medium instance`, enables the deployment of LLMs on [SageMaker JumpStart](https://aws.amazon.com/sagemaker/jumpstart/). These can be called with an API endpoint created by SageMaker, which we then use to build, experiment with, and tune for comparing Advanced RAG application techniques using [LangChain](https://www.langchain.com/). Additionally, we showcase how the [FAISS](https://github.com/facebookresearch/faiss) Embedding store can be utilized to archive and retrieve embeddings, integrating it into your RAG workflow. 

## Prerequisites

---
This Jupyter Notebook can be run on a t3.medium instance (ml.t3.medium). However, to deploy `Llama3-8B Text Generation` and `BGE Large En v1.5` models, you may need to request a quota increase. 

To request a quota increase, follow these steps:

1. Navigate to the [Service Quotas console](https://console.aws.amazon.com/servicequotas/).
2. Choose Amazon SageMaker.
3. Review your default quota for the following resources:
   - `ml.g5.12xlarge` for endpoint usage
   - `ml.g5.2xlarge` for endpoint usage
4. If needed, request a quota increase for these resources.

<div class="alert alert-block alert-warning"> 

<b>NOTE:</b> To make sure that you have enough quotas to support your usage requirements, it's a best practice to monitor and manage your service quotas. Requests for Amazon EC2 service quota increases are subject to review by AWS engineering teams. Also, service quota increase requests aren't immediately processed when you submit a request. After your request is processed, you receive an email notification.
</div>

### Changing instance type
---
Models are supported on the following instance types:

 - Llama3-8B Text Generation: `ml.g5.2xlarge`, `ml.g5.4xlarge`, `ml.g5.8xlarge`, `ml.g5.12xlarge`, `ml.g5.24xlarge`, `ml.g5.48xlarge`, and `ml.p4d.24xlarge`
 - BGE Large En v1.5: `ml.g5.2xlarge`, `ml.c6i.xlarge`,`ml.g5.4xlarge`, `ml.g5.8xlarge`, `ml.p3.2xlarge`, and `ml.g4dn.2xlarge`

By default, the JumpStartModel class selects a default instance type available in your region. If you would like to use a different instance type, you can do so by specifying instance type in the JumpStartModel class.

`my_model = JumpStartModel(model_id=model_id, instance_type="ml.g5.12xlarge")`

### Local setup (Optional):
---

For a local server, follow these steps to execute this jupyter notebook:

1. **Configure AWS CLI**: Configure [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) with your AWS credentials. Run `aws configure` and enter your AWS Access Key ID, AWS Secret Access Key, AWS Region, and default output format.

2. **Install required libraries**: Install the necessary Python libraries for working with SageMaker, such as [sagemaker](https://github.com/aws/sagemaker-python-sdk/), [boto3](https://github.com/boto/boto3), and others. You can use a Python environment manager like [conda](https://docs.conda.io/en/latest/) or [virtualenv](https://virtualenv.pypa.io/en/latest/) to manage your Python packages in your preferred IDE (e.g. [Visual Studio Code](https://code.visualstudio.com/)).

3. **Create an IAM role for SageMaker**: Create an AWS Identity and Access Management (IAM) role that grants your user [SageMaker permissions](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html). 

By following these steps, you can set up a local Jupyter Notebook environment capable of deploying machine learning models on Amazon SageMaker using the appropriate IAM role for granting the necessary permissions.

## Contents
---

1. [Requirements](#Requirements)
2. [Model Deployment](#Model-Deployment)
3. [Setup LangChain](#Setup-LangChain)
4. [Data Preparation](#Data-Preparation)
5. [Question Answering with LangChain Vector Store Wrapper](#Question-Answering-with-LangChain-Vector-Store-Wrapper)
7. [Improving RAG Responses for Long Documents with Langchain](#Improving-RAG-Responses-for-Long-Documents-with-Langchain)
8. [Conclusion](#Conclusion)
9. [Clean Up Resources](#Clean-Up-Resources)

## Requirements
---

1. Create an Amazon SageMaker Notebook Instance - [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
    - For Notebook Instance type, choose `ml.t3.medium`.
2. For Select Kernel, choose [conda_python3](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-prepare.html).
3. Install the required packages.

<div class="alert alert-block alert-info"> 

<b>NOTE:

- </b> For <a href="https://aws.amazon.com/sagemaker/studio/" target="_blank">Amazon SageMaker Studio</a>, select Kernel "<span style="color:green;">Python 3 (ipykernel)</span>".

- For <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html" target="_blank">Amazon SageMaker Studio Classic</a>, select Image "<span style="color:green;">Base Python 3.0</span>" and Kernel "<span style="color:green;">Python 3</span>".

</div>

To run this notebook you would need to install the following dependencies:

In [ ]:
%%writefile requirements.txt
langchain==0.1.14
pypdf==4.1.0
faiss-cpu==1.8.0
boto3==1.34.58
sqlalchemy==2.0.29

In [ ]:
!pip install -U -r requirements.txt --quiet

<div class="alert alert-block alert-warning"> 

<b>NOTE:</b>

Before proceeding, please verify that you have the correct version of the SQLAlchemy library installed. This notebook requires SQLAlchemy >= 2.0.0.

To check your installed SQLAlchemy version, you can run the following code:

```python
import sqlalchemy
print(sqlalchemy.__version__)
```

If the version displayed is less than 2.0.0, and you have already installed the correct version using `pip`, you may need to "<span style="color:green;">restart</span>" or "<span style="color:green;">shutdown</span>" the Jupyter Notebook kernel to load the updated library.

To restart the kernel, go to the "Kernel" menu and select "Restart Kernel". If that doesn't work, try shutting down the notebook completely and relaunching it.

Restarting or shutting down the kernel will resolve any dependency issues and ensure that the correct SQLAlchemy version is loaded.

If you haven't installed SQLAlchemy >= 2.0.0 yet, you can do so by running the following command in your terminal or command prompt:

```
pip install sqlalchemy>=2.0.29
```

Once the installation is complete, restart or shutdown the Jupyter Notebook kernel as described above.

</div>

In [ ]:
import sqlalchemy
print(sqlalchemy.__version__)

In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
try:
    import sagemaker
except ImportError:
    !pip install sagemaker --quiet

## Model Deployment
---

Deploy `Llama 3 8B Instruct` LLM model on Amazon SageMaker JumpStart:

In [ ]:
# Import the JumpStartModel class from the SageMaker JumpStart library
from sagemaker.jumpstart.model import JumpStartModel

In [ ]:
# Specify the model ID for the HuggingFace Llama 3 8b Instruct LLM model
model_id = "meta-textgeneration-llama-3-8b-instruct"
accept_eula = True
model = JumpStartModel(model_id=model_id)
# predictor = model.deploy(accept_eula=accept_eula)

Deploy `BGE Large En` embedding model on Amazon SageMaker JumpStart:

In [ ]:
# Specify the model ID for the HuggingFace BGE Large EN Embedding model
model_id = "huggingface-sentencesimilarity-bge-large-en-v1-5"
text_embedding_model = JumpStartModel(model_id=model_id)
# embedding_predictor = text_embedding_model.deploy()

## Setup LangChain
---

In [ ]:
import json
import sagemaker

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

Get endpoint names from predictors.

In [ ]:
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name
llm_endpoint_name = "meta-textgeneration-llama-3-8b-instruct-2024-05-24-00-31-15-408"
embedding_endpoint_name = "hf-sentencesimilarity-bge-large-en-v1-5-2024-05-24-00-07-17-320"

Transform input and output data to proccess API calls for`Llama 3 8B Instruct` on Amazon SageMaker

In [ ]:
from typing import Dict

class Llama38BContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 1000,
                "top_p": 0.9,
                "temperature": 0.6,
                "stop": ["<|eot_id|>"],
            },
        }
        input_str = json.dumps(
            payload,
        )
        #print(input_str)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        #print(response_json)
        content = response_json["generated_text"].strip()
        return content

Instantiate the LLM with SageMaker and LangChain

In [ ]:
# Instantiate the content handler for Llama3-8B
llama_content_handler = Llama38BContentHandler()

# Setup for using the Llama3-8B model with SageMaker Endpoint
llm = SagemakerEndpoint(
     endpoint_name=llm_endpoint_name,
     region_name=region, 
     model_kwargs={"max_new_tokens": 1024, "top_p": 0.9, "temperature": 0.7},
     content_handler=llama_content_handler
 )

Transform input and output data to proccess API calls for`BGE Large En` on Amazon SageMaker

In [ ]:
from typing import List

class BGEContentHandlerV15(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            text_inputs (list[str]): A list of input text strings to be processed.
            model_kwargs (Dict): Additional keyword arguments to be passed to the endpoint.
               Possible keys and their descriptions:
               - mode (str): Inference method. Valid modes are 'embedding', 'nn_corpus', and 'nn_train_data'.
               - corpus (str): Corpus for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - top_k (int): Top K for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - queries (list[str]): Queries for Nearest Neighbor. Required when mode is 'nn_corpus' or 'nn_train_data'.
        Returns:
            The transformed bytes input.
        """
        input_str = json.dumps(
            {
                "text_inputs": text_inputs,
                **model_kwargs
            }
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]

Instantiate the embedding model with SageMaker and LangChain

In [ ]:
bge_content_handler = BGEContentHandlerV15()
sagemaker_embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=embedding_endpoint_name,
    region_name=region,
    model_kwargs={"mode": "embedding"},
    content_handler=bge_content_handler,
)

## Data Preparation
---

Let's first download some of the files to build our document store.

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

In [ ]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf',
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf',
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/f965e5c3-fded-45d3-bbdb-f750f156dcc9.pdf',
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/336d8745-ea82-40a5-9acc-1a89df23d0f3.pdf'
]

filenames = [
    'AMZN-2024-10-K-Annual-Report.pdf',
    'AMZN-2023-10-K-Annual-Report.pdf',
    'AMZN-2022-10-K-Annual-Report.pdf',
    'AMZN-2021-10-K-Annual-Report.pdf'
]

metadata = [
    dict(year=2024, source=filenames[0]),
    dict(year=2023, source=filenames[1]),
    dict(year=2022, source=filenames[2]),
    dict(year=2021, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

If you take a look into the Amazon 10-Ks, the first 4 pages are all the very similar and may skew the responses if you they are kept in the embeddings. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 10-K (first 4 pages) and overwrite them as processed files.

In [ ]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

# Iterate over each PDF file
for idx, local_pdf in enumerate(local_pdfs):
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    
    if idx == 0:
        # Keep the first 4 pages for the first document
        for pagenum in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[pagenum]
            pdf_writer.add_page(page)
    else:
        # Remove the first 4 pages for other documents
        for pagenum in range(4, len(pdf_reader.pages)):
            page = pdf_reader.pages[pagenum]
            pdf_writer.add_page(page)

    # Write the modified content to a new file
    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

After downloading we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 512 tokens, which roughly translates to ~2000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [ ]:
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]

    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
)

docs = text_splitter.split_documents(documents)
print(docs[100])

Before we are proceeding we are looking into some interesting statistics regarding the document preprocessing we just performed:

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)

print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')

We had 4 PDF documents which have been split into smaller ~500 chunks.

Now we can see how a sample embedding would look like for one of those chunks.

In [ ]:
sample_embedding = np.array(sagemaker_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

This can be easily done using [FAISS](https://github.com/facebookresearch/faiss) implementation inside [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html) which takes  input the embeddings model and the documents to create the entire vector store. Using the Index Wrapper we can abstract away most of the heavy lifting such as creating the prompt, getting embeddings of the query, sampling the relevant documents and calling the LLM. [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation) helps us with that.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    sagemaker_embeddings,
)
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

## Question Answering with LangChain Vector Store Wrapper
---

We use the wrapper provided by LangChain which wraps around the Vector Store and takes input the LLM. This wrapper performs the following steps behind the scences:

- Takes input the question
- Create question embedding
- Fetch relevant documents
- Stuff the documents and the question into a prompt
- Invoke the model with the prompt and generate the answer in a human readable manner.

*Note: In this example we are using `Llama 3 8B Instruct` as the LLM under Amazon SageMaker, this particular model performs best if the inputs are provided under `<|begin_of_text|><|start_header_id|>system<|end_header_id|>`, `{{system_message}}`, `<|eot_id|><|start_header_id|>user<|end_header_id|>`, `{{user_message}}`, and the model is requested to generate an output after `<|eot_id|><|start_header_id|>assistant<|end_header_id|>`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.*

In [ ]:
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
{query}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["query"]
)

In [ ]:
query = "How did AWS perform in 2021?"

In [ ]:
answer = wrapper_store_faiss.query(question=PROMPT.format(query=query), llm=llm)
print(answer)

We can ask another question.

In [ ]:
query_2 = "How much square footage did Amazon have in North America in 2023?"

In [ ]:
answer = wrapper_store_faiss.query(question=PROMPT.format(query=query_2), llm=llm)
print(answer)

### Regular Retriever Chain
---
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the help of [RetrievalQA](https://docs.smith.langchain.com/cookbook/hub-examples/retrieval-qa-chain) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/) which can be specific to the model.

In [ ]:
from langchain.chains import RetrievalQA

prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

This is a conversation between an AI assistant and a Human.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
#### Context ####
{context}
#### End of Context ####

Question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

Let's start asking questions:

In [ ]:
query = "How did AWS perform in 2023?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "What are some of the risk factors associated to Amazon?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "Was Amazon involved in any lawsuits in 2022? What were they?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "What was Amazon's revenue in 2021?"

result = qa({"query": query})

print(result['result'])

print(f"\n{result['source_documents']}")

## Improving RAG Responses for Long Documents with Langchain

Let’s look at a more advanced RAG option with the help of ParentDocumentRetriever. When working with document retrieval, you may encounter a trade-off between storing small chunks of a document for accurate embeddings and larger documents to preserve more context. The parent document retriever strikes that balance by splitting and storing small chunks of data. Parent document retrievers provide LLMs with a twofold advantage: the specificity of child document embeddings for precise and relevant information retrieval, coupled with the invocation of parent documents for response generation, which enriches the LLM’s outputs with a layered and thorough context.

### Parent Document Retriever Chain
---

In this scenario, let's have a look at a more advanced rag option with the help of [ParentDocumentRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever). When working with document retrieval, you may encounter a trade-off between storing small chunks of a document for accurate embeddings and larger documents to preserve more context. The `ParentDocumentRetriever` strikes that balance by splitting and storing small chunks of data. 

The `ParentDocumentRetriever` uses an [InMemoryStore](https://api.python.langchain.com/en/v0.1.4/storage/langchain.storage.in_memory.InMemoryBaseStore.html) to store and manage the parent documents. By working with both parent and child documents, this approach aims to balance accurate embeddings with contextual information, providing more meaningful and relevant retrieval results.

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

Sometimes, the full documents can be too big to want to retrieve them as is. In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks. We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

In [ ]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The vectorstore to use to index the child chunks
vectorstore_faiss = FAISS.from_documents(
    child_splitter.split_documents(documents),
    sagemaker_embeddings,
)

# The storage layer for the parent documents
store = InMemoryStore()

In [ ]:
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore_faiss,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
retriever.add_documents(documents, ids=None)

Let’s now call the vector store search functionality - we should see that it returns small chunks (since we’re storing the small chunks).

In [ ]:
sub_docs = vectorstore_faiss.similarity_search("What was Amazon's revenue in 2021?")

In [ ]:
len(sub_docs[0].page_content)

In [ ]:
print(sub_docs[0].page_content)

Let’s now retrieve from the overall retriever. This should return large documents - since it returns the documents where the smaller chunks are located.

In [ ]:
retrieved_docs = retriever.get_relevant_documents("What was Amazon's revenue in 2021?")

In [ ]:
len(retrieved_docs[0].page_content)

In [ ]:
print(retrieved_docs[0].page_content)

Now, let's initialize the chain using the `ParentDocumentRetriever`. We will pass the prompt in via the chain_type_kwargs argument.

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

Let's start asking questions:

In [ ]:
query = "How did AWS perform in 2023?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "What are some of the risk factors associated to Amazon?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "Was Amazon involved in any lawsuits in 2022? What were they?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "What was the net sales change from 2022 to 2023?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

## Conclusion
---

Congratulations on completing the advanced retrieval augmented generation with `Llama3 8b`! Through this notebook, you were able to learn how to leverage the power of `Llama3 8b` with the precision of retrieval methods from `LangChain`. While building our QnA application, we are able to see that in contexts like detailing AWS’s transition from a simple service to a complex, multi-billion-dollar entity, or explaining Amazon's strategic successes, the Regular Retriever Chain lacks the precision the more sophisticated techniques offer, leading to less targeted information. For customers in industries such as HCLS, Telecommunications, and FSI who are looking to implement RAG in their applications,  the limitations of the Regular Retriever Chain in providing precision, avoiding redundancy, and effectively compressing information make them less suited to fulfilling these needs compared to the Parent Document Retriever that is able to distill vast amounts of information into the concentrated, impactful insights that customers need, while helping improve price performance.

In the above implementation of Advanced RAG based Question Answering we have explored the following concepts and how to implement them using Amazon SageMaker JumpStart and it's LangChain integration.

- Deploying models on Amazon SageMaker JumpStart
- Setting up `Llama3-8b` and `BGE Large En v1.5` with LangChain
- Loading documents of different kind and generating embeddings to create a vector store
- Retrieving documents to the question using the following approaches from LangChain
    - Regular Retrieval Chain
    - Parent Document Retriever Chain
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner

### Take-aways
---
- Experiment with different retrieval techniques
- Leverage `Llama3-8b` and `BGE Large En v1.5` models available under Amazon SageMaker JumpStart
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

## Clean Up Resources
---

In [ ]:
# Delete resources
llm_predictor.delete_model()
llm_predictor.delete_endpoint()
embedding_predictor.delete_model()
embedding_predictor.delete_endpoint()

# Thank You!